In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset

In [7]:
num_classes = 4
batchsize = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
learning_rate = 0.001
num_epochs = 50

In [8]:
class EEGDataset(Dataset):
    def __init__(self, data_path, labels_path):
        self.data = np.load(data_path)
        self.labels = np.load(labels_path)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get the EEG data and corresponding label
        eeg = torch.tensor(self.data[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return eeg, label

# Example usage
train_dataset = EEGDataset("eeg_dataset/train_epochs.npy", "eeg_dataset/train_labels.npy")
val_dataset = EEGDataset("eeg_dataset/val_epochs.npy", "eeg_dataset/val_labels.npy")
test_dataset = EEGDataset("eeg_dataset/test_epochs.npy", "eeg_dataset/test_labels.npy")

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batchsize, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batchsize, shuffle=False)

print(len(train_loader))
# Check one batch
for eeg_batch, label_batch in train_loader:
    print("EEG Batch Shape:", eeg_batch.shape)  # (batch_size, 14, 640)
    print("Label Batch Shape:", label_batch.shape)  # (batch_size,)
    break


27
EEG Batch Shape: torch.Size([16, 1076])
Label Batch Shape: torch.Size([16])


In [9]:
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.1)
        self.dropout3 = nn.Dropout(p=0.1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(64)
        
        self.l1 = nn.Linear(input_size, 512)
        self.l2 = nn.Linear(512, 256)
        self.l3 = nn.Linear(256, 64)
        self.l4 = nn.Linear(64, num_classes)
        
    def forward(self, x):
        x = self.l1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        
        x = self.l2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout2(x)

        x = self.l3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout3(x)
        
        x = self.l4(x)
        return x
        

In [10]:
model = MLP(1076, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for eeg, labels in train_loader:
        
        eeg = eeg.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        output = model(eeg)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Validation loop
    model.eval()
    val_accuracy = 0
    with torch.no_grad():
        for eeg, labels in val_loader:
            eeg = eeg.to(device)
            labels = labels.to(device)
            
            output = model(eeg)
            pred = output.argmax(dim=1, keepdim=True)
            val_accuracy += pred.eq(labels.view_as(pred)).sum().item()

    val_accuracy /= len(val_loader.dataset)
    print(f'Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}, Validation Accuracy: {val_accuracy * 100}%')


Epoch 1, Training Loss: 1.3848482017163877, Validation Accuracy: 40.65934065934066%
Epoch 2, Training Loss: 1.2781710315633703, Validation Accuracy: 56.043956043956044%
Epoch 3, Training Loss: 1.1571742826037936, Validation Accuracy: 67.03296703296702%
Epoch 4, Training Loss: 1.0620528260866802, Validation Accuracy: 75.82417582417582%
Epoch 5, Training Loss: 1.0126917053152014, Validation Accuracy: 85.71428571428571%
Epoch 6, Training Loss: 0.935917059580485, Validation Accuracy: 84.61538461538461%
Epoch 7, Training Loss: 0.8738032976786295, Validation Accuracy: 87.91208791208791%
Epoch 8, Training Loss: 0.8443008815800702, Validation Accuracy: 86.81318681318682%
Epoch 9, Training Loss: 0.790680295891232, Validation Accuracy: 91.20879120879121%
Epoch 10, Training Loss: 0.7558234002855089, Validation Accuracy: 91.20879120879121%
Epoch 11, Training Loss: 0.7149097853236728, Validation Accuracy: 96.7032967032967%
Epoch 12, Training Loss: 0.6689813688949302, Validation Accuracy: 92.3076923

In [12]:
# test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for eeg, labels in test_loader:
        eeg = eeg.to(device)
        labels = labels.to(device)
        outputs = model(eeg)
        
        # # print(outputs)
        # print(torch.max(outputs, 1))
        # print("ACC")
        # print(labels)
        # break
        
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions==labels).sum().item()
        
    acc = 100 * (n_correct/n_samples)
    print(f'accuracy = {acc}')

accuracy = 90.0
